# 2D Hidden Linear Function

Based on [Quantum advantage with shallow circuits](https://arxiv.org/pdf/1704.00690.pdf) by Sergey Bravyi, David Gosset and Robert König.

## The problem

Given: $A \in \mathbb{F}_2^{n \times n}, b \in \mathbb{F}_2^n$.

Define function $q : \mathbb{F}_2^n \to \mathbb{Z}_4 $:

$$q(x) = (2 x^T A x + b^T x) \% 4$$ 

Also define

$$\mathcal{L}_q = \{x \in  \mathbb{F}_2^n : q(x \oplus y) = (q(x) + q(y)) \% 4 ~~ \forall y \in \mathbb{F}_2^n \}$$

Then there exists such $z \in \mathbb{F}_2^n$, that

$$q(x) = 2 z^T x \forall x \in \mathcal{L}_q$$

Problems is, given $A$ and $b$, to find $z$.

First, let's implement classical bruteforce solution.

In [198]:
import numpy as np

class HiddenLinearFunctionProblem:
    def __init__(self, A, b):
        self.n = A.shape[0]
        assert A.shape == (self.n, self.n)
        assert b.shape == (self.n, )
        for i in range(self.n):
            for j in range(i+1):
                assert A[i][j] == 0, 'A[i][j] can be 1 only if i<j'
        
        self.A = A
        self.b = b
        
        self._all_vectors = [np.array([(m>>i)%2 for i in range(self.n)]) for m in range(2**self.n)]
        self.L = [x for x in self._all_vectors if self.vector_in_L(x)]
        self.all_zs = [z for z in self._all_vectors if self.is_z(z)]

    def vector_in_L(self, x):
        assert len(self._all_vectors) == 2**self.n
        for y in self._all_vectors:
            if self.q( (x + y)%2 ) != (self.q(x) + self.q(y))%4:
                return False
        return True
                                        
    def q(self, x):
        assert x.shape == (self.n, )
        return (2 * (x @ self.A @ x) + (self.b @ x)) % 4
    
    def is_z(self, z):
        assert len(self._all_vectors) == 2**self.n
        for x in self.L:
            if self.q(x) != 2 * ((z @ x) % 2):
                return False
        return True

In [199]:
def random_problem(n, seed=None):
    if seed is not None:
        np.random.seed(seed) 
    A = np.random.randint(0, 2, size=(n,n))
    for i in range(n):
        for j in range(i+1):
            A[i][j] = 0
    b = np.random.randint(0, 2, size=n)
    problem = HiddenLinearFunctionProblem(A, b)
    print('Generated problem where %d vectors of %d are solutions.' % (len(problem.all_zs), 2**n))
    return problem
        
def find_interesting_problem(n, min_L_size):
    for _ in range(1000):
        problem = random_problem(n)
        if len(problem.L) >= min_L_size:
            return problem
    return None

find_interesting_problem(5, 4)

Generated problem where 32 vectors of 32 are solutions.
Generated problem where 16 vectors of 32 are solutions.
Generated problem where 32 vectors of 32 are solutions.
Generated problem where 16 vectors of 32 are solutions.
Generated problem where 8 vectors of 32 are solutions.


In [204]:
import cirq

def solve_problem(problem, print_circuit=False):
    # Building the circuit.
    qubits = cirq.LineQubit.range(problem.n)
    circuit = cirq.Circuit()
    
    def add_hadamards(c):
        c += cirq.Moment([cirq.H(qubits[i]) for i in range(problem.n)])
            
    add_hadamards(circuit)
    
    for i in range(problem.n):
        for j in range(i+1, problem.n):
            if problem.A[i][j] == 1:
                circuit += cirq.CZ(qubits[i], qubits[j])
    
    S_moment = cirq.Moment()
    for i in range(problem.n):
        if problem.b[i] == 1:
            S_moment += cirq.ZPowGate(exponent=-0.5).on(qubits[i])
    circuit += S_moment
            
    add_hadamards(circuit)
        
        
    if print_circuit:
        print(circuit)
        sim = cirq.Simulator()
        #print(sim.compute_amplitudes(circuit, bitstrings=[i for i in range(2**problem.n)]))
    
    for i in range(problem.n):
        circuit += cirq.measure(qubits[i], key=str(i))
    
    
    # Sampling the cirquit.
    sim = cirq.Simulator()
    result = sim.simulate(circuit)
    z = np.array([result.measurements[str(i)][0] for i in range(problem.n)])
    return z

def test_problem(problem):
    ans = {}
    attempts_count = 100
    ok_count = 0
    for _ in range(attempts_count):
        z = solve_problem(problem)
        if problem.is_z(z):
            ok_count += 1 
        z_string = ''.join([str(x) for x in z])
        if z_string in ans:
            ans[z_string] += 1
        else:
            ans[z_string] = 1
    print('Success %d of %d times' % (ok_count, attempts_count))
    return ans



In [205]:
A = np.array([[0, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 1],
       [0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])
b = np.array([1, 0, 0, 1, 1, 1, 1, 1])
problem = HiddenLinearFunctionProblem(A, b)

print(len(problem.L), len(problem.all_zs))
solve_problem(problem, print_circuit=True)
test_problem(problem)

16 16
              ┌──┐   ┌──┐   ┌──┐           ┌──┐   ┌──┐   ┌──┐
0: ───H───@────@──────@─────────────────────────────────────────────────S^-1───H───
          │    │      │
1: ───H───@────┼@─────┼@─────@─────────────────────────────────────────────────H───
               ││     ││     │
2: ───H────────@┼─────┼┼─────┼@────@───@────@──────@───────────────────────────H───
                │     ││     ││    │   │    │      │
3: ───H─────────┼─────@┼─────┼@────┼───┼────┼@─────┼──────@─────────────S^-1───H───
                │      │     │     │   │    ││     │      │
4: ───H─────────@──────┼─────┼─────@───┼────┼┼─────┼@─────┼@────@───────S^-1───H───
                       │     │         │    ││     ││     ││    │
5: ───H────────────────@─────┼─────────@────┼@─────┼@─────┼┼────┼───@───S^-1───H───
                             │              │      │      ││    │   │
6: ───H──────────────────────┼──────────────@──────┼──────┼@────┼───┼───S^-1───H───
                             │          

{'00100101': 8,
 '01011000': 10,
 '00011101': 8,
 '11000111': 5,
 '10101000': 6,
 '10000010': 10,
 '01110010': 4,
 '10010000': 7,
 '00110111': 7,
 '01100000': 6,
 '10111010': 5,
 '11111111': 5,
 '11010101': 3,
 '01001010': 6,
 '11101101': 9,
 '00001111': 1}

In [219]:
p = random_problem(10)
test_problem(p)

Generated problem where 256 vectors of 1024 are solutions.
Success 100 of 100 times


{'0111110111': 1,
 '1010111001': 1,
 '0111111011': 1,
 '0100000001': 1,
 '0101010111': 1,
 '1001110100': 2,
 '0010100110': 1,
 '1000011100': 1,
 '1010111011': 2,
 '1111010100': 1,
 '1010000101': 1,
 '1111101111': 1,
 '0001010111': 1,
 '1101001101': 2,
 '0001011001': 1,
 '1001110011': 1,
 '1010111100': 1,
 '0001010101': 1,
 '1000100111': 2,
 '0110011111': 1,
 '0101100010': 1,
 '1110111001': 2,
 '0011110010': 1,
 '1010111110': 1,
 '1001001101': 1,
 '0100000011': 1,
 '1001000100': 1,
 '0000111010': 1,
 '0111001001': 2,
 '1110000101': 1,
 '0101011110': 1,
 '0000110011': 1,
 '0110100110': 1,
 '0110101010': 1,
 '1110001100': 2,
 '0111110010': 1,
 '1111100110': 2,
 '0111110000': 1,
 '1001110110': 1,
 '1111100001': 1,
 '1000101110': 1,
 '1110110101': 1,
 '0000110100': 1,
 '0010101101': 1,
 '1111010110': 1,
 '0001010000': 1,
 '0111001100': 1,
 '1001111000': 1,
 '1100101100': 1,
 '0000001101': 1,
 '0111001011': 1,
 '1110110111': 1,
 '1110001001': 1,
 '1111100011': 2,
 '1101000011': 1,
 '10110111